In [ ]:
import os
import subprocess
import re

def extract_pgs_subtitles(source_file, output_directory):
    """
    Extracts PGS subtitles from a video file and saves them as a .sup file.
    """
    # Determine the correct subtitle stream index dynamically
    subtitle_stream_index = "0:4"
    
    if subtitle_stream_index is None:
        print(f"No English PGS subtitle stream found in {source_file}.")
        return

    # Create the output directory if it does not exist
    os.makedirs(output_directory, exist_ok=True)

    # File path for the extracted subtitles
    subtitle_file = os.path.join(output_directory, 'subtitles.sup')

    # FFmpeg command to extract PGS subtitles
    extract_command = [
        'ffmpeg',
        '-y',  # Overwrite output files without asking
        '-i', source_file,
        '-map', subtitle_stream_index,
        '-c', 'copy',  # Copy the subtitle stream without re-encoding
        subtitle_file
    ]

    try:
        subprocess.run(extract_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Subtitles extracted successfully to {subtitle_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error extracting subtitles from {source_file}: {e}")

In [ ]:
def process_videos(source_dir, destination_dir):
    """
    Process all video files in the source directory to extract PGS subtitles.
    """
    # Define the range of seasons (e.g., Season 1 to Season 7)
    seasons = sorted([f"Season {i}" for i in range(1, 8)])

    # Loop through each season folder
    for season in seasons:
        season_path = os.path.join(source_dir, season)

        # Check if the season directory exists
        if not os.path.isdir(season_path):
            print(f"Directory {season_path} does not exist, skipping.")
            continue

        # Create corresponding destination directory for the season
        destination_season_path = os.path.join(destination_dir, season)
        os.makedirs(destination_season_path, exist_ok=True)

        # Get sorted list of files in the season folder
        files = sorted(os.listdir(season_path))

        # Process each file in the season folder
        for filename in files:
            # Look for complete .mkv files
            if filename.endswith('.mkv') and not filename.endswith('.mkv.part'):

                pattern = r"S\d{2}E\d{2} - [^()]+"

                match = re.search(pattern, filename)
                if match:
                    modified_filename = match.group(0)
                    # print(episode_title)  # Output: S01E01 - Fundraiser
                # else:
                    # print("Pattern not found in the filename.")

                source_file = os.path.join(season_path, filename)

                # Create a directory for the episode
                output_directory = os.path.join(destination_season_path, modified_filename.strip())

                # Ensure the output directory exists
                os.makedirs(output_directory, exist_ok=True)

                # Extract PGS subtitles
                extract_pgs_subtitles(source_file, output_directory)


In [ ]:
# Specify the source and destination directories
source_directory = "/media/coof/standardHDD/Veep"  # Replace with the path to your directory containing seasons
destination_directory = "/media/coof/standardHDD/Veep2"

# Call the function to burn subtitles
process_videos(source_directory, destination_directory)
